In [1]:
import sys
!{sys.executable} -m pip install csv_to_sqlite

You are using pip version 9.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sqlite3
conn = sqlite3.connect('air.db')
c = conn.cursor()
c.execute("select * from ast")

In [3]:
import csv_to_sqlite
from pathlib import Path
Path('air.db').touch()
import pandas as pd
asthma = pd.read_csv("Asthma & Air Quality Data.csv")
asthma.to_sql('ast', conn, if_exists='append', index = False)

d = c.execute('''SELECT * FROM ast''').fetchall()

from sqlalchemy import create_engine 
  
# SQLAlchemy connectable 
cnx = create_engine('sqlite:///air.db').connect() 
  
# table named 'contacts' will be returned as a dataframe. 
df = pd.read_sql_table('ast', 'sqlite:///air.db')
df['Asthma ER visits']=df['Asthma ER visits'].str.replace(',','')
df['Asthma ER visits'] = df['Asthma ER visits'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 8 columns):
Borough                      1134 non-null object
Geography - UHF 42           1134 non-null object
Latitude                     1134 non-null float64
Longitude                    1134 non-null float64
Air Quality Indicator        1134 non-null object
Mean, mcg per cubic meter    1134 non-null float64
Age Group                    1134 non-null object
Asthma ER visits             1134 non-null float64
dtypes: float64(4), object(4)
memory usage: 71.0+ KB


/Users/TinaShahid/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [4]:
import folium
import pandas as pd

from folium.plugins import MarkerCluster

asthma = pd.read_csv("Asthma & Air Quality Data.csv")
asthma.head()

,Borough,Geography - UHF 42,Latitude,Longitude,Air Quality Indicator,"Mean, mcg per cubic meter",Age Group,Asthma ER visits
0,Bronx,Crotona - Tremont,40.846229,-73.890437,Fine Particulate Matter (PM2.5),8.2,0 to 17 years,"3,161"
1,Bronx,Fordham - Bronx Pk,40.866630,-73.883030,Fine Particulate Matter (PM2.5),7.6,0 to 17 years,"3,120"
2,Bronx,High Bridge - Morrisania,40.844632,-73.922661,Fine Particulate Matter (PM2.5),8.2,0 to 17 years,"3,559"
3,Bronx,Hunts Point - Mott Haven,40.820281,-73.883187,Fine Particulate Matter (PM2.5),8.3,0 to 17 years,"2,486"
4,Bronx,Kingsbridge - Riverdale,40.886612,-73.899685,Fine Particulate Matter (PM2.5),7.5,0 to 17 years,336


In [5]:

map = folium.Map(location=[40.7128, -74.0060], zoom_start = 10)

# Display the map
map.save('mymap.html')
map

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 8 columns):
Borough                      1134 non-null object
Geography - UHF 42           1134 non-null object
Latitude                     1134 non-null float64
Longitude                    1134 non-null float64
Air Quality Indicator        1134 non-null object
Mean, mcg per cubic meter    1134 non-null float64
Age Group                    1134 non-null object
Asthma ER visits             1134 non-null float64
dtypes: float64(4), object(4)
memory usage: 71.0+ KB


In [6]:
map.choropleth(geo_data="map.geojson",
             data= df, # my dataset SQL lite dataset here
             #df['Asthma ER visits']=df['Asthma ER visits'].astype(str),
             columns=['Geography - UHF 42', 'Asthma ER visits'], # geography is here for matching the geojson geography, er visits is the column that changes the color of er visits areas
             key_on='feature.properties.Geography', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2)
             #legend_name='Asthma ER Visits'

map.save('mymap.html')
map

In [7]:
# add a marker for every record in the filtered data, use a clustered view
marker_cluster = MarkerCluster().add_to(map) # create marker clusters

for i in range(df.shape[0]):
    location = [df['Latitude'][i],df['Longitude'][i]]
    tooltip = "Borough: {}<br> Asthma ER visits:{}<br> Mean, mcg per cubic meter:{}<br>".format(df["Geography - UHF 42"][i],
                                                                                                df['Asthma ER visits'][i],
                                                                                                df['Mean, mcg per cubic meter'][i])
    
    folium.Marker(location, # adding more details to the popup screen using HTML
                  popup="Click here",
                  tooltip=tooltip).add_to(marker_cluster)
map
